## Smart Virtual Hiring Assistant
# This code implements a conversational AI bot for HR teams to filter candidates
# Features: Resume parsing, skill-matching, sentiment analysis, candidate summary
# Tech: GPT-3.5 (mocked), LangChain, Flask (simulated), Python
# Designed to run in Google Colab


In [1]:
# Install required packages
!pip install langchain openai nltk pandas numpy scikit-learn

import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import random
from IPython.display import display, HTML

# Download NLTK data
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Mock GPT-3.5 API (for demo purposes, replace with actual OpenAI API in production)
class MockOpenAI:
    def __init__(self):
        self.responses = {
            "Tell me about your experience with Python.": [
                "I've worked with Python for 3 years, building web apps with Flask and data pipelines with Pandas.",
                "I have 5 years of Python experience, focusing on machine learning with scikit-learn and TensorFlow."
            ],
            "What is your greatest strength?": [
                "My greatest strength is problem-solving; I enjoy tackling complex challenges with creative solutions.",
                "I'm highly adaptable, quickly learning new tools and technologies to deliver results."
            ]
        }

    def generate_response(self, prompt):
        return random.choice(self.responses.get(prompt, ["I'm sorry, I don't have a response for that."]))

In [3]:
# Initialize mock GPT API
llm = MockOpenAI()

# Simulated resume data (replace with actual resume parsing logic)
resume_data = {
    "candidate_1": {
        "name": "John Doe",
        "skills": ["Python", "Flask", "SQL", "Pandas"],
        "experience": "3 years as a Data Engineer at TechCorp, building ETL pipelines.",
        "education": "BS Computer Science, XYZ University"
    },
    "candidate_2": {
        "name": "Jane Smith",
        "skills": ["Python", "Django", "Machine Learning", "TensorFlow"],
        "experience": "5 years as a Machine Learning Engineer at AI Innovations.",
        "education": "MS Data Science, ABC University"
    }
}

In [4]:
# Predefined job requirements
job_requirements = {
    "role": "Data Engineer",
    "required_skills": ["Python", "SQL", "Pandas", "Flask"],
    "optional_skills": ["Machine Learning", "AWS"],
    "experience": "3+ years"
}

In [5]:
# Initialize sentiment analyzer
sid = SentimentIntensityAnalyzer()

In [6]:
# Resume parsing and skill-matching
def parse_resume(resume):
    skills = resume.get("skills", [])
    experience = resume.get("experience", "")
    return skills, experience

In [7]:
def calculate_resume_score(skills, required_skills, optional_skills):
    vectorizer = TfidfVectorizer()
    all_skills = [", ".join(skills), ", ".join(required_skills + optional_skills)]
    tfidf_matrix = vectorizer.fit_transform(all_skills)
    score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return round(score * 100, 2)

In [8]:
# Candidate interview simulation
def conduct_interview(candidate_id, questions):
    responses = {}
    sentiments = {}
    for question in questions:
        response = llm.generate_response(question)
        responses[question] = response
        sentiment = sid.polarity_scores(response)
        sentiments[question] = sentiment['compound']
    return responses, sentiments

In [13]:
# Generate candidate summary
def generate_candidate_summary(candidate_id, resume, responses, sentiments, resume_score):
    summary = f"""
    <h2>Candidate Summary: {resume['name']}</h2>
    <p><b>Resume Score:</b> {resume_score}%</p>
    <p><b>Education:</b> {resume['education']}</p>
    <p><b>Experience:</b> {resume['experience']}</p>
    <p><b>Skills:</b> {', '.join(resume['skills'])}</p>
    <h3>Interview Responses:</h3>
    """
    for question, response in responses.items():
        sentiment = "Positive" if sentiments[question] > 0 else "Neutral" if sentiments[question] == 0 else "Negative"
        summary += f"<p><b>{question}</b><br>{response}<br><i>Sentiment: {sentiment}</i></p>"

    # Auto-generated feedback
    feedback = "The candidate shows strong alignment with the role." if resume_score > 80 else "The candidate has relevant skills but may need further evaluation."
    summary += f"<h3>Feedback:</h3><p>{feedback}</p>"
    return summary

In [14]:
# Main execution
def main():
    questions = [
        "Tell me about your experience with Python.",
        "What is your greatest strength?"
    ]

    for candidate_id, resume in resume_data.items():
        # Parse resume
        skills, experience = parse_resume(resume)

        # Calculate resume score
        resume_score = calculate_resume_score(skills, job_requirements["required_skills"], job_requirements["optional_skills"])

        # Conduct interview
        responses, sentiments = conduct_interview(candidate_id, questions)

        # Generate summary
        summary = generate_candidate_summary(candidate_id, resume, responses, sentiments, resume_score)

        # Display results
        display(HTML(summary))

# Run the assistant
main()

In [15]:
# Adjustable tone simulation (for demo purposes)
def adjust_tone(text, tone="professional"):
    if tone == "friendly":
        return text.replace("shows strong alignment", "is a great fit").replace("may need further evaluation", "could shine with a bit more review")
    return text

In [16]:
# Example of adjusting tone
sample_feedback = "The candidate shows strong alignment with the role."
print("Professional Tone:", sample_feedback)
print("Friendly Tone:", adjust_tone(sample_feedback, tone="friendly"))

Professional Tone: The candidate shows strong alignment with the role.
Friendly Tone: The candidate is a great fit with the role.
